In [18]:
import anndata as ad
adata = ad.read_h5ad('resources/grn_benchmark/evaluation_data/replogle_sc.h5ad')
adata.obs['is_control'] = adata.obs['perturbation']=='non-targeting'
        
mask_sample =  adata.obs['is_control']
mask_sample.sum()

0

In [ ]:
adata = ad.read_h5ad('resources/grn_benchmark/inference_data/replogle_rna.h5ad')
adata[adata.obs['is_control']]

/home/jnourisa/miniconda3/envs/gimme/lib/python3.10/site-packages/anndata/_core/aligned_df.py:68: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


(585, 8244)